In [1]:
#Pip commands if needed
#!pip install optax==0.1.4
#!pip install dynamax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.model_selection import train_test_split
from dynamax.hidden_markov_model import CategoricalHMM
import numpy as np
import pandas as pd
import jax.numpy as jnp
import jax.random as jr
from jax import vmap

In [3]:
#Get data and work with all possible 
!unzip music_data_cleaned.zip

Archive:  music_data_cleaned.zip
replace Cleaned Data/Taube.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace Cleaned Data/index/Busoni song index.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
folder_path = '/content/Cleaned Data/'
Beethoven = np.load(folder_path+'Beethoven.npy', allow_pickle = True)
Handel = np.load(folder_path+'Haendel.npy', allow_pickle = True)
Chopin = np.load(folder_path+'Chopin.npy', allow_pickle = True)

comps_list = [Beethoven,Handel,Chopin]

In [5]:
channel_names = ['tick','time','note value','channel','velocity','duration']

composers = [[],[],[]]
for i,composer in enumerate(comps_list):
    for song in composer:
        df = pd.DataFrame(song,columns=channel_names)
        df.drop(['tick','time','channel','velocity'],axis=1,inplace=True)
        df["duration"][df["duration"] < 1e-12] = 0 #filter small note durations
        df["duration"][df["duration"] > 2e3] = 2e3 #filter big note durations
        df.ffill(inplace=True)
        composers[i].append(df.to_numpy())

Beethoven = composers[0]
Handel = composers[1]
Chopin = composers[2]
comps_list = [Beethoven,Handel,Chopin]

In [6]:
# Label the data
label_dict = {0:"Bach", 1:"Beethoven", 2:"Handel", 3:"Chopin"}

Beethoven_lbl = np.vstack((Beethoven,1*np.ones(len(Beethoven))))
Handel_lbl = np.vstack((Handel,2*np.ones(len(Handel))))
Chopin_lbl = np.vstack((Chopin,3*np.ones(len(Chopin))))


/usr/local/lib/python3.9/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [7]:
def print_params(params):
    jnp.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
    print("initial probs:")
    print(params.initial.probs)
    print("transition matrix:")
    print(params.transitions.transition_matrix)
    print("emission probs:")
    print(params.emissions.probs) 

In [8]:
# Concatenate into one big data array and train-test-split
data = np.concatenate((Beethoven_lbl, Handel_lbl, Chopin_lbl), axis=1)

X, Y = data[0, :], data[1, :]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [ ]:
# GaussianHMM fitting
params_list = []
num_unique_notes = 96
num_unique_durations = 2092
num_states = 3
num_emmisions = 2
num_classes = num_unique_notes*num_unique_durations

for i,p in enumerate(X_train):
    if (np.isnan(p).any()) or (np.isinf(p).any()):
        print("NaN or Inf")
        print(np.isnan(p).any())
        print(np.isinf(p).any())
        print(" at i =", i)
        continue
    hmm = CategoricalHMM(num_states,num_emmisions,num_classes)
    params, props = hmm.initialize(method="prior")
    params, log_probs = hmm.fit_em(params,props,p,num_iters=10)
    params_list.append(params)

In [ ]:
t_matrix_list = []
for params in params_list:
    t_matrix_list.append(params.transitions.transition_matrix)